In [4]:
install.packages("remotes")
remotes::install_github("cellgeni/sceasy")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done




rprojroot  (2.0.3  -> 2.0.4 ) [CRAN]
Rcpp       (1.0.11 -> 1.0.12) [CRAN]
withr      (2.5.1  -> 3.0.0 ) [CRAN]
rlang      (1.1.1  -> 1.1.3 ) [CRAN]
jsonlite   (1.8.7  -> 1.8.8 ) [CRAN]
reticulate (1.32.0 -> 1.36.0) [CRAN]


Installing 6 packages: rprojroot, Rcpp, withr, rlang, jsonlite, reticulate

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



── R CMD build ─────────────────────────────────────────────────────────────────
✔  checking for file ‘/tmp/RtmpbcuksJ/remotesf524393ce26/cellgeni-sceasy-c1c0bf9/DESCRIPTION’
─  preparing ‘sceasy’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘sceasy_0.0.7.tar.gz’
   


In [1]:
library(sceasy)
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(H5weaver))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(stringr))
library(parallel)

Loading required package: reticulate



In [167]:
# # Helper function to compare two folder for similar/different files

# # Specify the paths of the two folders to compare
# folder1 <- "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female"
# folder2 <- "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Male_v2"

# # List files in folder1
# files1 <- dir(folder1)

# # List files in folder2
# files2 <- dir(folder2)

# # Find matching files
# matching_files <- intersect(files1, files2)

# # Find files in folder1 that are not in folder2
# files_only_in_folder1 <- setdiff(files1, files2)

# # Find files in folder2 that are not in folder1
# files_only_in_folder2 <- setdiff(files2, files1)

# # Print results
# cat("Files matching:\n")
# print(matching_files)

# cat("\nFiles only in folder1:\n")
# print(files_only_in_folder1)

# cat("\nFiles only in folder2:\n")
# print(files_only_in_folder2)

Files matching:
 [1] "Adaptive NK cell_Y1D0.csv"                    
 [2] "ASDC_Y1D0.csv"                                
 [3] "BaEoMaP cell_Y1D0.csv"                        
 [4] "C1Q+ CD16 monocyte_Y1D0.csv"                  
 [5] "CD14+ cDC2_Y1D0.csv"                          
 [6] "CD27- effector B cell_Y1D0.csv"               
 [7] "CD27+ effector B cell_Y1D0.csv"               
 [8] "CD4 MAIT_Y1D0.csv"                            
 [9] "CD56bright NK cell_Y1D0.csv"                  
[10] "CD8 MAIT_Y1D0.csv"                            
[11] "CD8aa_Y1D0.csv"                               
[12] "CD95 memory B cell_Y1D0.csv"                  
[13] "cDC1_Y1D0.csv"                                
[14] "CLP cell_Y1D0.csv"                            
[15] "CM CD4 T cell_Y1D0.csv"                       
[16] "CM CD8 T cell_Y1D0.csv"                       
[17] "CMP cell_Y1D0.csv"                            
[18] "Core CD14 monocyte_Y1D0.csv"                  
[19] "Core CD16 monocyte_Y1D0.

In [2]:
meta_data=read.csv('/home/jupyter/IH-A-Aging-Analysis-Notebooks/Qiuyu-Notebooks/scRNA/hise_meta_data_2024-01-23_fixed.csv')
meta_data<-meta_data %>% filter(Covid_exlcusion=='no',sample.visitName %in% c('Flu Year 1 Day 0'))
meta_data<-meta_data %>% mutate (Flu_Year=case_when(Sample_Draw_date>as.Date('2019-6-30')&Sample_Draw_date<as.Date('2020-6-29')~'2019',
                                         Sample_Draw_date>as.Date('2020-6-30')&Sample_Draw_date<as.Date('2021-6-29')~'2020',
                                         Sample_Draw_date>as.Date('2021-6-30')&Sample_Draw_date<as.Date('2022-6-29')~'2021',
                                        ))

In [3]:
t(table(meta_data$subject.subjectGuid,meta_data$Flu_Year))

      
       BR1001 BR1002 BR1003 BR1004 BR1005 BR1006 BR1007 BR1008 BR1009 BR1010
  2019      1      1      1      1      1      1      1      1      1      1
  2020      0      0      0      0      0      0      0      0      0      0
      
       BR1011 BR1012 BR1013 BR1014 BR1015 BR1016 BR1017 BR1018 BR1019 BR1021
  2019      1      1      1      1      0      0      0      1      0      0
  2020      0      0      0      0      1      1      1      0      1      1
      
       BR1023 BR1024 BR1025 BR1026 BR1028 BR1030 BR1031 BR1032 BR1033 BR1037
  2019      0      0      0      0      0      0      0      0      1      0
  2020      1      1      1      1      1      1      1      1      0      1
      
       BR1040 BR1041 BR1042 BR1043 BR1044 BR1045 BR1047 BR1048 BR1049 BR1050
  2019      0      0      0      0      0      0      0      0      0      0
  2020      1      1      1      1      1      1      1      1      1      1
      
       BR1051 BR1052 BR1054 BR1056 BR1057

In [4]:
## add an additional column
meta_data$facet = paste(meta_data$subject.biologicalSex, meta_data$cohort.cohortGuid, sep = "_")
meta_data$facet_2 <- paste(meta_data$facet, meta_data$CMV.IgG.Serology.Result.Interpretation, sep = "_")

In [5]:
head(meta_data)

,X.1,X,lastUpdated,sample.id,sample.bridgingControl,sample.sampleKitGuid,sample.visitName,sample.visitDetails,sample.drawDate,sample.daysSinceFirstVisit,⋯,Sample_Draw_date,Covid_Dose_1,Covid_Dose_2,Covid_exlcusion,BMI,file.name.downloaded,file.path,Flu_Year,facet,facet_2
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,1,2023-11-18T02:03:08.347Z,f499ff83-e513-4d24-a10f-151348269fff,false,KT00001,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-15,NA,NA,no,<25-BMI,B001-P1_PB00001-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/fec489f9-9a74-4635-aa91-d2bf09d1faec/B001-P1_PB00001-01_labeled.h5,2019,Female_BR1,Female_BR1_Negative
2,2,2,2023-11-18T02:03:08.347Z,750e90a9-a296-4f0f-969f-60225c2bca17,false,KT00002,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-15,2020-12-28,2021-01-18,no,<25-BMI,B001-P1_PB00002-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/7c0c7979-eebd-4aba-b5b2-6e76b4643623/B001-P1_PB00002-01_labeled.h5,2019,Male_BR1,Male_BR1_Negative
3,3,3,2023-11-18T02:03:08.347Z,2db6fb3f-e3f4-454b-891b-9b068541b51d,false,KT00003,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-15,2020-12-28,2021-01-16,no,<25-BMI,B001-P1_PB00003-01_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/40efd03a-cb2f-4677-af42-a056cbfe5a17/B001-P1_PB00003-01_labeled.h5,2019,Female_BR1,Female_BR1_Negative
4,4,4,2023-11-19T10:34:34.12Z,f04693c5-563c-4b5b-ae58-877d0d9ae2fe,false,KT00004,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-21,2021-04-16,2021-05-06,no,<25-BMI,B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/68fbcd34-1d63-461d-8195-df5b8dc61b31/B002-P1_PB00004-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019,Male_BR1,Male_BR1_Negative
5,5,5,2023-11-19T05:07:36.805Z,eb5b3a3d-002e-40a6-aa19-aa0e6a7fff8f,false,KT00006,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-23,2021-01-16,2021-02-26,no,<25-BMI,B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/ea8d98e9-e99e-4dc6-9e78-9866e0deac68/B002-P1_PB00006-01_2023-11-17T21:36:51.794326181Z_labeled.h5,2019,Female_BR1,Female_BR1_Negative
6,6,6,2023-11-18T02:03:08.347Z,f32ac061-0ede-4040-8d8c-35229ea5229c,false,KT00010,Flu Year 1 Day 0,N/A - Flu-Series Timepoint Only,2019-10-01T00:00:00Z,0,⋯,2019-10-28,2021-03-21,2021-04-18,no,>25-30BMI,B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,/home/jupyter/BRI_Analysis/scRNA/cache/237e41df-5616-4852-8a8f-b32835c9efcb/B078-P2_PB00010-02_2021-08-19T17:09:29.934849811Z_labeled.h5,2019,Male_BR1,Male_BR1_Negative


# Comapre BR1 vs BR2 in Male and Female separately

## Test 1 - with CMV sample size accounted for in Females

In [6]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)
dim(meta_data_subset)

[1] 51 39

In [8]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet_2 == "Female_BR2_Positive")
subset_BR1 <- filter(meta_data_subset, facet_2 == "Female_BR1_Negative")
subset_BR2_N <- filter(meta_data_subset, facet_2 == "Female_BR2_Negative")
subset_BR1_P <- filter(meta_data_subset, facet_2 == "Female_BR1_Positive")

length(subset_BR2$pbmc_sample_id) + length(subset_BR1$pbmc_sample_id) + length(subset_BR2_N$pbmc_sample_id) +length(subset_BR1_P$pbmc_sample_id)
length(subset_BR1_P$pbmc_sample_id)


[1] 51

[1] 9

In [134]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 11)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 12)
selected_BR2_N <- sample(unique(subset_BR2_N$pbmc_sample_id), 9)
selected_BR1_P <- sample(unique(subset_BR1_P$pbmc_sample_id), 9)

length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
length(selected_BR2_N)
length(selected_BR1_P)

#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1
selected_sample_ids_BR2
selected_BR2_N
selected_BR1_P

[1] 11

[1] 12

[1] 9

[1] 9

[1] "PB00362-01" "PB00350-01" "PB00529-01" "PB00621-01" "PB00023-05"
 [6] "PB00003-01" "PB00006-01" "PB00030-02" "PB00347-01" "PB00365-01"
[11] "PB00001-01" "PB00345-01"

[1] "PB00552-01" "PB00569-01" "PB00352-04" "PB00557-01" "PB00533-01"
 [6] "PB00387-01" "PB00599-01" "PB00602-01" "PB00530-01" "PB00537-01"
[11] "PB00368-07"

[1] "PB00353-04" "PB00501-01" "PB00394-02" "PB00526-01" "PB00377-01"
[6] "PB00545-01" "PB00334-01" "PB00568-01" "PB00393-01"

[1] "PB00031-05" "PB00012-01" "PB00349-01" "PB00022-01" "PB00575-01"
[6] "PB00525-01" "PB00341-01" "PB00166-01" "PB00532-01"

In [135]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1, selected_BR1_P, selected_BR2_N)

length(selected_sample_ids)
length(unique(selected_sample_ids))
#selected_sample_ids[duplicated(selected_sample_ids)] # check for duplicated if number of sample ids not same

[1] 41

[1] 41

character(0)

In [136]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [137]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

[1] 41

In [138]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_1/master_female_CMV_pseudobulk_tes1.rds")

   user  system elapsed 
494.604 322.614  23.108 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [139]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_1/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
1008.616 1618.449  948.607 

## Test 2 - with CMV sample size accounted for in Females

In [140]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)
dim(meta_data_subset)

[1] 51 39

In [142]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet_2 == "Female_BR2_Positive")
subset_BR1 <- filter(meta_data_subset, facet_2 == "Female_BR1_Negative")
subset_BR2_N <- filter(meta_data_subset, facet_2 == "Female_BR2_Negative")
subset_BR1_P <- filter(meta_data_subset, facet_2 == "Female_BR1_Positive")

length(unique(subset_BR1_P$pbmc_sample_id))
length(unique(subset_BR2_N$pbmc_sample_id))
length(subset_BR2$pbmc_sample_id) + length(subset_BR1$pbmc_sample_id) + length(subset_BR2_N$pbmc_sample_id) +length(subset_BR1_P$pbmc_sample_id)


[1] 9

[1] 9

[1] 51

In [143]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 11)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 12)
selected_BR2_N <- sample(unique(subset_BR2_N$pbmc_sample_id), 9)
selected_BR1_P <- sample(unique(subset_BR1_P$pbmc_sample_id), 9)

length(selected_sample_ids_BR2)
length(selected_sample_ids_BR1)
length(selected_BR2_N)
length(selected_BR1_P)

#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1
selected_sample_ids_BR2
selected_BR2_N
selected_BR1_P

[1] 11

[1] 12

[1] 9

[1] 9

[1] "PB00507-01" "PB00003-01" "PB00001-01" "PB00347-01" "PB00338-01"
 [6] "PB00345-01" "PB00365-01" "PB00621-01" "PB00023-05" "PB00529-01"
[11] "PB01425-01" "PB00366-01"

[1] "PB00387-01" "PB00541-01" "PB00557-01" "PB00530-01" "PB00552-01"
 [6] "PB00602-01" "PB00561-04" "PB00516-01" "PB00368-07" "PB00537-01"
[11] "PB00569-01"

[1] "PB00393-01" "PB00334-01" "PB00394-02" "PB00377-01" "PB00501-01"
[6] "PB00545-01" "PB00526-01" "PB00353-04" "PB00568-01"

[1] "PB00012-01" "PB00022-01" "PB00166-01" "PB00575-01" "PB00532-01"
[6] "PB00525-01" "PB00341-01" "PB00349-01" "PB00031-05"

In [144]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1, selected_BR1_P, selected_BR2_N)

length(selected_sample_ids)
length(unique(selected_sample_ids))
#selected_sample_ids[duplicated(selected_sample_ids)] # check for duplicated if number of sample ids not same

[1] 41

[1] 41

In [136]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [145]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

[1] 41

In [146]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_2/master_female_CMV_pseudobulk_test2.rds")


   user  system elapsed 
491.104 310.628  21.903 

In [6]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [147]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_2/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
1000.017 1614.312  966.694 

## Test 3 - with CMV sample size accounted for in Females

In [148]:
meta_data_subset<-meta_data%>% filter(subject.biologicalSex=='Female', sample.visitName %in% c('Flu Year 1 Day 0')) 
#head(meta_data_subset)
dim(meta_data_subset)

[1] 51 39

In [149]:
# Filter the data based on the criteria
subset_BR2 <- filter(meta_data_subset, facet_2 == "Female_BR2_Positive")
subset_BR1 <- filter(meta_data_subset, facet_2 == "Female_BR1_Negative")
subset_BR2_N <- filter(meta_data_subset, facet_2 == "Female_BR2_Negative")
subset_BR1_P <- filter(meta_data_subset, facet_2 == "Female_BR1_Positive")

length(unique(subset_BR1_P$pbmc_sample_id))
length(subset_BR2$pbmc_sample_id) + length(subset_BR1$pbmc_sample_id) + length(subset_BR2_N$pbmc_sample_id) +length(subset_BR1_P$pbmc_sample_id)


[1] 9

[1] 51

In [150]:
# Sample from the filtered subsets
selected_sample_ids_BR2 <- sample(unique(subset_BR2$pbmc_sample_id), 11)
selected_sample_ids_BR1 <- sample(unique(subset_BR1$pbmc_sample_id), 12)
selected_BR2_N <- sample(unique(subset_BR2_N$pbmc_sample_id), 9)
selected_BR1_P <- sample(unique(subset_BR1_P$pbmc_sample_id), 9)

length(selected_sample_ids_BR2)  #Female_BR2_Positive
length(selected_sample_ids_BR1)  #Female_BR1_Negative
length(selected_BR2_N)           #Female_BR2_Negative
length(selected_BR1_P)           #Female_BR1_Positive

#length(unique(selected_sample_ids_BR1$sample.id))
selected_sample_ids_BR1  #BR1_N
selected_sample_ids_BR2  #BR2_P
selected_BR2_N
selected_BR1_P


[1] 11

[1] 12

[1] 9

[1] 9

[1] "PB00003-01" "PB00366-01" "PB00338-01" "PB00347-01" "PB01425-01"
 [6] "PB00621-01" "PB00345-01" "PB00529-01" "PB00030-02" "PB00362-01"
[11] "PB00023-05" "PB00507-01"

[1] "PB00388-01" "PB00602-01" "PB00537-01" "PB00352-04" "PB00533-01"
 [6] "PB00552-01" "PB00368-07" "PB00016-01" "PB00561-04" "PB00387-01"
[11] "PB00599-01"

[1] "PB00394-02" "PB00526-01" "PB00353-04" "PB00377-01" "PB00545-01"
[6] "PB00393-01" "PB00568-01" "PB00501-01" "PB00334-01"

[1] "PB00166-01" "PB00349-01" "PB00532-01" "PB00022-01" "PB00012-01"
[6] "PB00525-01" "PB00575-01" "PB00341-01" "PB00031-05"

In [152]:
# Combine the sampled IDs
selected_sample_ids <- c(selected_sample_ids_BR2, selected_sample_ids_BR1, selected_BR1_P, selected_BR2_N)

length(selected_sample_ids)
length(unique(selected_sample_ids))
#selected_sample_ids[duplicated(selected_sample_ids)] # check for duplicated if number of sample ids not same

[1] 41

[1] 41

In [153]:
# #head(selected_sample_ids)
# length(unique(subset_BR2$pbmc_sample_id))
# length(unique(selected_sample_ids))
# #selected_sample_ids_BR2

In [154]:
# Filter the data for the selected sample IDs
meta_data_subset <- meta_data_subset[meta_data_subset$pbmc_sample_id %in% selected_sample_ids, ]
length(unique(meta_data_subset$pbmc_sample_id))
length(unique(meta_data_subset$pbmc_sample_id))

[1] 41

[1] 41

In [155]:
system.time(
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){
pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)
},mc.cores=60)
)

# Combine all pseudobulk objects into a single data frame
master_bulk <- do.call(cbind, bulk_list)

# Save master pseudobulk object
saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_3/master_female_CMV_pseudobulk_test3.rds")


   user  system elapsed 
449.516 306.198  23.000 

In [156]:
# # Combine all pseudobulk objects into a single data frame
# master_bulk <- do.call(cbind, bulk_list)

# # Save master pseudobulk object
# saveRDS(master_bulk, file = "/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/master_pseudobulk.rds")

In [157]:
system.time(
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('/home/jupyter/Myeloid_cells/files/filtered_gene_list_for_fluyr1D0.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  cohort.cohortGuid +
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("cohort.cohortGuid", "BR2", "BR1")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'BR1',log2FoldChange>0~'BR2'))
res$Year<-"Y1"
write.csv(res,paste0("/home/jupyter/IH-A-Aging-Analysis-Notebooks_old/Mansi_Notebooks/scRNA/DEG_Analysis/MS_Sex_DEG_analysis/DEG_cohort_splitby_sex_CSVfiles/Age_yr1_Day0_Female_CMV_Test_3/",celltype,"_Y1D0.csv"))

}
)

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Adaptive NK cell"


...

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 1 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



    user   system  elapsed 
 964.433 1565.503  919.355 